In [1]:
# Gru
import pandas as pd
import numpy as np
from dataset import LABELS
from sklearn.preprocessing import LabelEncoder
import random


In [2]:
from dataset import labels_only_detection_training, labels_only_detection_validation, labels_only_detection_full

In [3]:
from models import Gru

In [4]:
res = pd.DataFrame(columns=['gru_units', 'dropout_rate', 'learning_rate', 'epoch', 'batch', 'loss_max','accuracy_max','val_loss_max', 'val_accuracy_max'])
# res.columns = ['gru_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max']
for gru_unit in [64,128]:
    for epoch in [20, 30]:
        for lr in [0.001, 0.01, 0.1]:
            for batch in [64, 32, 128]:
                model = Gru(gru_units=gru_unit, dropout_rate=0.1, learning_rate=lr, num_classes=10, batch_size=batch, epoch=epoch)
                model.train(labels_only_detection_training, labels_only_detection_validation)
                res = np.concatenate((res, pd.DataFrame([[gru_unit,0.1, lr, epoch, batch, model.history.history['loss'][-1], model.history.history['accuracy'][-1], model.history.history['val_loss'][-1], model.history.history['val_accuracy'][-1]]], 
                                                                        columns=['gru_units', 'dropout_rate', 'learning_rate', 'epoch', 'batch', 'loss_max','accuracy_max','val_loss_max', 'val_accuracy_max'])))

Epoch 1/20
660/660 [==============================] - ETA: 0s - loss: 2.0643 - accuracy: 0.3122
Epoch 1: val_accuracy improved from -inf to 0.58673, saving model to models\gru.h5
660/660 [==============================] - 178s 147ms/step - loss: 2.0643 - accuracy: 0.3122 - val_loss: 1.1387 - val_accuracy: 0.5867
Epoch 2/20
660/660 [==============================] - ETA: 0s - loss: 1.1974 - accuracy: 0.5764
Epoch 2: val_accuracy improved from 0.58673 to 0.67404, saving model to models\gru.h5
660/660 [==============================] - 81s 122ms/step - loss: 1.1974 - accuracy: 0.5764 - val_loss: 0.9452 - val_accuracy: 0.6740
Epoch 3/20
660/660 [==============================] - ETA: 0s - loss: 0.9642 - accuracy: 0.6688
Epoch 3: val_accuracy improved from 0.67404 to 0.74078, saving model to models\gru.h5
660/660 [==============================] - 102s 154ms/step - loss: 0.9642 - accuracy: 0.6688 - val_loss: 0.7850 - val_accuracy: 0.7408
Epoch 4/20
660/660 [==============================] -

KeyboardInterrupt: 

In [5]:
pd.DataFrame(res,columns=['gru_units', 'dropout_rate', 'learning_rate', 'epoch', 'batch', 'loss_max','accuracy_max','val_loss_max', 'val_accuracy_max']).to_pickle('results\\gru_wyniki_only_known.pkl')

In [52]:
res = pd.read_pickle('results\\again_search_for_hparams.pkl')

In [50]:
res[res.val_accuracy_max == res.val_accuracy_max.max()]

,gru_units,dropout_rate,epoch,batch,loss_max,accuracy_max,val_loss_max,val_accuracy_max
2,64.0,0.2,20.0,128.0,0.017513,0.996447,0.002362,0.999709
10,64.0,0.2,30.0,64.0,0.016528,0.997584,0.007348,0.999709
19,128.0,0.2,20.0,64.0,0.01021,0.99801,0.002979,0.999709


In [5]:
model = Gru(128,0.2,20,64,0.01,(39,44), 30, 'models\\gru_mod.h5')

In [6]:
model.train(label_detection_validation)

Epoch 1/20
106/107 [============================>.] - ETA: 0s - loss: 1.9617 - accuracy: 0.5392
Epoch 1: accuracy improved from -inf to 0.53913, saving model to models\gru_mod.h5
107/107 [==============================] - 22s 155ms/step - loss: 1.9615 - accuracy: 0.5391
Epoch 2/20
106/107 [============================>.] - ETA: 0s - loss: 1.4505 - accuracy: 0.6169
Epoch 2: accuracy improved from 0.53913 to 0.61695, saving model to models\gru_mod.h5
107/107 [==============================] - 15s 141ms/step - loss: 1.4497 - accuracy: 0.6169
Epoch 3/20
106/107 [============================>.] - ETA: 0s - loss: 1.2981 - accuracy: 0.6288
Epoch 3: accuracy improved from 0.61695 to 0.62871, saving model to models\gru_mod.h5
107/107 [==============================] - 17s 155ms/step - loss: 1.2993 - accuracy: 0.6287
Epoch 4/20
106/107 [============================>.] - ETA: 0s - loss: 1.1761 - accuracy: 0.6439
Epoch 4: accuracy improved from 0.62871 to 0.64342, saving model to models\gru_mod.h5

In [9]:
pd.read_csv('sample_submission.csv')

,fname,label
0,clip_000044442.wav,unknown
1,clip_0000adecb.wav,unknown
2,clip_0000d4322.wav,unknown
3,clip_0000fb6fe.wav,unknown
4,clip_0001d1559.wav,unknown
...,...,...
158533,clip_fffe49419.wav,unknown
158534,clip_ffff2fb36.wav,no
158535,clip_ffff90f56.wav,no
158536,clip_ffff98589.wav,no


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional, TimeDistributed, BatchNormalization
from dataset import LABELS
from sklearn.preprocessing import LabelEncoder
from dataset import label_detection_training, label_detection_validation, silence_detection_training, silence_detection_validation
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataset import TensorflowDataset

In [3]:
train = pd.read_pickle('extracted_features\\features_training.pkl')
y_train = np.array([x[1] for x in train])
labels = list(np.unique(y_train))
train = TensorflowDataset('extracted_features\\features_training.pkl', labels=labels).dataset
train = train.shuffle(len(train), reshuffle_each_iteration=True)
val = TensorflowDataset('extracted_features\\features_validation.pkl', labels=labels).dataset
val = val.shuffle(len(val), reshuffle_each_iteration=True)

In [8]:
# Set your model's hyperparameters
results = pd.DataFrame(columns=['gru_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])

from tensorflow.keras.layers import GRU, Dropout, Dense, Bidirectional, TimeDistributed, BatchNormalization, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D

for gru_units in [64, 128]:
    for dropout_rate in [0.2, 0.4]:
            for epoch in [ 10, 20]:
                  for batch_size in [32, 64]:
                        print(f'gru_units: {gru_units}, dropout_rate: {dropout_rate}, epoch: {epoch}, batch_size: {batch_size}')
                        num_classes = len(np.unique(labels))  # Number of unique classes in your dataset

                        input_shape = (39, 44)


                        model = Sequential([
                            Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=input_shape),
                            BatchNormalization(),
                            MaxPooling1D(pool_size=2),
                            Dropout(dropout_rate),
                            Bidirectional(GRU(gru_units, return_sequences=True)),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Bidirectional(GRU(gru_units, return_sequences=True)),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            GRU(gru_units, return_sequences=True),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            GRU(gru_units, return_sequences=True),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(256, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(128, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(64, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            GlobalMaxPooling1D(),
                            Dense(512, activation='relu'),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Dense(256, activation='relu'),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Dense(128, activation='relu'),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Dense(num_classes, activation='softmax')
                        ])


                        # Compile the model
                        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                        # Train the model with your training set and validate it with your validation set
                        history = model.fit(train.batch(batch_size), epochs=epoch, validation_data=val.batch(batch_size))

                        results = np.concatenate((results, pd.DataFrame([[gru_units, dropout_rate, epoch, batch_size, history.history['loss'][-1], history.history['loss'], history.history['accuracy'][-1], history.history['accuracy'], history.history['val_loss'][-1], history.history['val_loss'], history.history['val_accuracy'][-1], history.history['val_accuracy']]], 
                                                                        columns=['gru_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])), axis=0)


gru_units: 64, dropout_rate: 0.2, epoch: 10, batch_size: 32
Epoch 1/10
1811/1811 [==============================] - 129s 63ms/step - loss: 2.6652 - accuracy: 0.2528 - val_loss: 1.9334 - val_accuracy: 0.4479
Epoch 2/10
1811/1811 [==============================] - 109s 60ms/step - loss: 1.6147 - accuracy: 0.5191 - val_loss: 1.2439 - val_accuracy: 0.6325
Epoch 3/10
1811/1811 [==============================] - 115s 63ms/step - loss: 1.2631 - accuracy: 0.6300 - val_loss: 1.1999 - val_accuracy: 0.6531
Epoch 4/10
1811/1811 [==============================] - 120s 66ms/step - loss: 1.0962 - accuracy: 0.6830 - val_loss: 0.9321 - val_accuracy: 0.7339
Epoch 5/10
1811/1811 [==============================] - 136s 75ms/step - loss: 0.9915 - accuracy: 0.7112 - val_loss: 0.9044 - val_accuracy: 0.7445
Epoch 6/10
1811/1811 [==============================] - 155s 85ms/step - loss: 0.9212 - accuracy: 0.7350 - val_loss: 0.8960 - val_accuracy: 0.7417
Epoch 7/10
1811/1811 [==============================] - 15

KeyboardInterrupt: 

In [9]:
results_f = pd.DataFrame(results)

In [10]:
results_f.to_pickle('results\\model_gru_final_version.pkl')

In [11]:
results_f

,0,1,2,3,4,5,6,7,8,9,10,11
0,64,0.2,10,32,0.746932,"[2.665213108062744, 1.6146924495697021, 1.2631...",0.785215,"[0.2528011202812195, 0.5191202163696289, 0.629...",0.765207,"[1.9333562850952148, 1.2438899278640747, 1.199...",0.789938,"[0.447925865650177, 0.6325389742851257, 0.6531..."
1,64,0.2,10,64,0.722944,"[2.7639663219451904, 1.5994480848312378, 1.273...",0.791395,"[0.23180773854255676, 0.521537184715271, 0.624...",0.747728,"[1.6140880584716797, 1.2849770784378052, 1.108...",0.789497,"[0.5142688751220703, 0.610914945602417, 0.6681..."
2,64,0.2,20,32,0.564152,"[2.6195595264434814, 1.5511832237243652, 1.253...",0.838389,"[0.26476529240608215, 0.5392158627510071, 0.63...",0.648608,"[1.784652590751648, 1.3517918586730957, 1.0938...",0.830097,"[0.44851428270339966, 0.596057653427124, 0.689..."
3,64,0.2,20,64,0.538738,"[2.697845935821533, 1.6010715961456299, 1.2770...",0.842222,"[0.2499525249004364, 0.5225730538368225, 0.621...",0.641245,"[1.5733102560043335, 1.1619383096694946, 1.054...",0.818035,"[0.5483965873718262, 0.6631362438201904, 0.691..."
4,64,0.4,10,32,1.121856,"[3.5523223876953125, 2.7618539333343506, 2.129...",0.687948,"[0.052138183265924454, 0.1850905567407608, 0.3...",2.346172,"[3.1456222534179688, 2.78471302986145, 2.16078...",0.31524,"[0.08046483993530273, 0.10311856120824814, 0.2..."
5,64,0.4,10,64,1.185146,"[3.6519148349761963, 3.016111135482788, 2.6201...",0.668422,"[0.04808107390999794, 0.12409578263759613, 0.2...",2.227695,"[3.3813705444335938, 3.035633087158203, 2.6361...",0.30759,"[0.06663724780082703, 0.09546925872564316, 0.2..."


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dropout, Dense, Bidirectional, TimeDistributed, BatchNormalization, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D

results = pd.DataFrame(columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])

for gru_units in [64, 128, 256]:
    for dropout_rate in [0.2, 0.4, 0.6]:
        for batch_size in [32, 64, 128]:
            for epochs in [10, 20, 30]:
                num_classes = len(y_train_encoded)  # Number of unique classes in your dataset
                input_shape = (39, 44)

                model = Sequential([
                    Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=input_shape),
                    BatchNormalization(),
                    MaxPooling1D(pool_size=2),
                    Dropout(dropout_rate),
                    Bidirectional(GRU(gru_units, return_sequences=True)),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    Bidirectional(GRU(gru_units, return_sequences=True)),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    GRU(gru_units, return_sequences=True),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    GRU(gru_units, return_sequences=True),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    TimeDistributed(Dense(256, activation='relu')),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    TimeDistributed(Dense(128, activation='relu')),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    TimeDistributed(Dense(64, activation='relu')),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    GlobalMaxPooling1D(),
                    Dense(512, activation='relu'),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    Dense(256, activation='relu'),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    Dense(128, activation='relu'),
                    BatchNormalization(),
                    Dropout(dropout_rate),
                    Dense(num_classes, activation='softmax')
                ])

                # Compile the model
                model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                # Train the model with your training set and validate it with your validation set
                epochs = epochs
                batch_size = batch_size
                history = model.fit(X_train, y_train_encoded, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val_encoded))

                results = np.concatenate((results, pd.DataFrame([[gru_units, dropout_rate, epochs, batch_size, history.history['loss'][-1], history.history['loss'], history.history['accuracy'][-1], history.history['accuracy'], history.history['val_loss'][-1], history.history['val_loss'], history.history['val_accuracy'][-1], history.history['val_accuracy']]], 
                                                                    columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])), axis=0)



Epoch 1/10
214/214 [==============================] - 29s 69ms/step - loss: 6.9579 - accuracy: 0.0613 - val_loss: 3.6162 - val_accuracy: 0.0596
Epoch 2/10
214/214 [==============================] - 12s 56ms/step - loss: 2.8994 - accuracy: 0.1918 - val_loss: 2.5313 - val_accuracy: 0.2439
Epoch 3/10
214/214 [==============================] - 13s 60ms/step - loss: 2.4038 - accuracy: 0.2900 - val_loss: 2.0998 - val_accuracy: 0.3616
Epoch 4/10
214/214 [==============================] - 12s 57ms/step - loss: 2.0500 - accuracy: 0.3930 - val_loss: 1.8843 - val_accuracy: 0.4451
Epoch 5/10
214/214 [==============================] - 13s 62ms/step - loss: 1.8284 - accuracy: 0.4449 - val_loss: 1.6862 - val_accuracy: 0.4878
Epoch 6/10
214/214 [==============================] - 12s 56ms/step - loss: 1.7079 - accuracy: 0.4755 - val_loss: 1.6471 - val_accuracy: 0.5157
Epoch 7/10
214/214 [==============================] - 13s 61ms/step - loss: 1.6029 - accuracy: 0.5110 - val_loss: 1.7111 - val_accuracy:

In [4]:
from keras.optimizers import Adam

opt = Adam(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 37, 128)           17024     
                                                                 
 batch_normalization (BatchN  (None, 37, 128)          512       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 18, 128)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 18, 128)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 18, 256)          198144    
 l)                                                              
                                                        

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
results.to_pickle('results\\model_gru')